In [24]:
api_key = 'PKQQI82T93OUPMF0W38Z'
api_secret = 'xqbQleavhspJWJSoL0nMlkOVDAjQUQMQDs76hb1u'

In [77]:
import alpaca_trade_api as tradeapi
import numpy as np 
import time
from alpaca_trade_api.rest import REST, TimeFrame
from datetime import datetime, timedelta


In [32]:
SEC_KEY = api_secret
PUB_KEY = api_key
BASE_URL = 'https://paper-api.alpaca.markets' # This is the base URL for paper trading
api = tradeapi.REST(key_id= PUB_KEY, secret_key=SEC_KEY, base_url=BASE_URL) # For real trading, don't enter a base_url



In [33]:
account = api.get_account()
print(account)

Account({   'account_blocked': False,
    'account_number': 'PA37HXB34JZF',
    'accrued_fees': '0',
    'buying_power': '199999.34',
    'cash': '99999.67',
    'created_at': '2022-07-27T17:34:56.879028Z',
    'crypto_status': 'ACTIVE',
    'currency': 'USD',
    'daytrade_count': 1,
    'daytrading_buying_power': '0',
    'equity': '99999.67',
    'id': 'e670d409-2c96-4c09-a973-03b561e32e08',
    'initial_margin': '0',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '99999.67',
    'pattern_day_trader': False,
    'pending_transfer_in': '0',
    'portfolio_value': '99999.67',
    'regt_buying_power': '199999.34',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '0',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
    'transfers_blocked': False})


In [28]:

# Buy a stock
api.submit_order(
  symbol='SPY', # Replace with the ticker of the stock you want to buy
  qty=1,
  side='buy',
  type='market', 
  time_in_force='gtc' # Good 'til cancelled
)



Order({   'asset_class': 'us_equity',
    'asset_id': 'b28f4066-5c6d-479b-a2af-85dc1a8f16fb',
    'canceled_at': None,
    'client_order_id': 'db76c298-c1e4-45b8-9b5c-b6240dfd4978',
    'created_at': '2022-07-27T17:36:43.387928091Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': 'd5643b87-a01a-437e-a243-92bf7552f417',
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'source': None,
    'status': 'pending_new',
    'stop_price': None,
    'submitted_at': '2022-07-27T17:36:43.382583771Z',
    'subtag': None,
    'symbol': 'SPY',
    'time_in_force': 'gtc',
    'trail_percent': None,
    'trail_price': None,
    'type': 'market',
    'updated_at': '2022-07-27T17:36:43.387973131Z'})

In [30]:
# Sell a stock(Just change side to 'sell')
api.submit_order(
  symbol='SPY',
  qty=1,
  side='sell',
  type='market',
  time_in_force='gtc'
)

Order({   'asset_class': 'us_equity',
    'asset_id': 'b28f4066-5c6d-479b-a2af-85dc1a8f16fb',
    'canceled_at': None,
    'client_order_id': '259792c3-5c6e-455f-9bd7-826f94658257',
    'created_at': '2022-07-27T17:37:28.689531951Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': '63eccb37-eb73-4049-bc0a-4fedce08785b',
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'sell',
    'source': None,
    'status': 'pending_new',
    'stop_price': None,
    'submitted_at': '2022-07-27T17:37:28.68910582Z',
    'subtag': None,
    'symbol': 'SPY',
    'time_in_force': 'gtc',
    'trail_percent': None,
    'trail_price': None,
    'type': 'market',
    'updated_at': '2022-07-27T17:37:28.68959132Z'})

In [80]:
symb = 'SPY'
pos_held = False
while True:
    # now = datetime.now()
    # my_time = now - timedelta(hours=48)
    # my_time = my_time.isoformat()
    # print(my_time)

    print("")
    print("Checking Price")
    
    market_data = api.get_bars(symb, TimeFrame.Minute, limit=5) # Get one bar object for each of the past 5 minutes
    print(market_data)
    close_list = [] # This array will store all the closing prices from the last 5 minutes
    for bar in market_data:
        close_list.append(bar.c) # bar.c is the closing price of that bar's time interval

    print(close_list)
    close_list = np.array(close_list, dtype=np.float64) # Convert to numpy array
    ma = np.mean(close_list)
    last_price = close_list[4] # Most recent closing price

    print("Moving Average: " + str(ma))
    print("Last Price: " + str(last_price))

    
    if ma + 0.1 < last_price and not pos_held: # If MA is more than 10cents under price, and we haven't already bought
        print("Buy")
        api.submit_order(
            symbol=symb,
            qty=1,
            side='buy',
            type='market',
            time_in_force='gtc'
        )
        pos_held = True
    elif ma - 0.1 > last_price and pos_held: # If MA is more than 10cents above price, and we already bought
        print("Sell")
        api.submit_order(
            symbol=symb,
            qty=1,
            side='sell',
            type='market',
            time_in_force='gtc'
        )
        pos_held = False
     
    time.sleep(61)
    


Checking Price
[Bar({   'c': 394.57,
    'h': 394.6,
    'l': 394.28,
    'n': 16,
    'o': 394.5,
    't': '2022-07-27T08:00:00Z',
    'v': 3166,
    'vw': 394.48669}), Bar({   'c': 394.71,
    'h': 394.71,
    'l': 394.7,
    'n': 3,
    'o': 394.7,
    't': '2022-07-27T08:02:00Z',
    'v': 247,
    'vw': 394.704008}), Bar({   'c': 394.79,
    'h': 394.79,
    'l': 394.74,
    'n': 11,
    'o': 394.74,
    't': '2022-07-27T08:03:00Z',
    'v': 418,
    'vw': 394.759234}), Bar({   'c': 394.78,
    'h': 394.79,
    'l': 394.74,
    'n': 8,
    'o': 394.74,
    't': '2022-07-27T08:04:00Z',
    'v': 1493,
    'vw': 394.769953}), Bar({   'c': 394.82,
    'h': 394.83,
    'l': 394.82,
    'n': 4,
    'o': 394.83,
    't': '2022-07-27T08:05:00Z',
    'v': 356,
    'vw': 394.821292})]
[394.57, 394.71, 394.79, 394.78, 394.82]
Moving Average: 394.734
Last Price: 394.82


KeyboardInterrupt: 

In [91]:
import alpaca_trade_api as tradeapi
import numpy as np
import time

symb = "TSLA"
pos_held = False
hours_to_test = 6

print("Checking Price")
market_data = api.get_bars(symb, TimeFrame.Minute, limit=(60 * hours_to_test)) # Pull market data from the past 60x minutes

close_list = []
for bar in market_data:
    close_list.append(bar.c)



print("Open: " + str(close_list[0]))
print("Close: " + str(close_list[60 * hours_to_test - 1]))


close_list = np.array(close_list, dtype=np.float64)
startBal = 200 # Start out with 2000 dollars
balance = startBal
buys = 0
sells = 0



for i in range(4, 60 * hours_to_test): # Start four minutes in, so that MA can be calculated
    ma = np.mean(close_list[i-4:i+1])
    last_price = close_list[i]

    print("Moving Average: " + str(ma))
    print("Last Price: " + str(last_price))

    if ma + 0.1 < last_price and not pos_held:
        print("Buy")
        balance -= last_price
        pos_held = True
        buys += 1
    elif ma - 0.1 > last_price and pos_held:
        print("Sell")
        balance += last_price
        pos_held = False
        sells += 1
    print(balance)
    time.sleep(0.01)

print("")
print("Buys: " + str(buys))
print("Sells: " + str(sells))

if buys > sells:
    balance += close_list[60 * hours_to_test - 1] # Add back your equity to your balance
    

print("Final Balance: " + str(balance))

print("Profit if held: " + str(close_list[60 * hours_to_test - 1] - close_list[0]))
print("Profit from algorithm: " + str(balance - startBal))

Checking Price
Open: 789
Close: 813.55
Moving Average: 790.78
Last Price: 792.2
Buy
-592.2
Moving Average: 791.622
Last Price: 793.21
-592.2
Moving Average: 792.422
Last Price: 794.0
-592.2
Moving Average: 793.402
Last Price: 795.9
-592.2
Moving Average: 794.162
Last Price: 795.5
-592.2
Moving Average: 794.722
Last Price: 795.0
-592.2
Moving Average: 794.88
Last Price: 794.0
Sell
201.79999999999995
Moving Average: 794.9300000000001
Last Price: 794.25
201.79999999999995
Moving Average: 794.55
Last Price: 794.0
201.79999999999995
Moving Average: 794.25
Last Price: 794.0
201.79999999999995
Moving Average: 794.05
Last Price: 794.0
201.79999999999995
Moving Average: 794.15
Last Price: 794.5
Buy
-592.7
Moving Average: 794.2479999999999
Last Price: 794.74
-592.7
Moving Average: 794.348
Last Price: 794.5
-592.7
Moving Average: 794.048
Last Price: 792.5
Sell
199.79999999999995
Moving Average: 793.848
Last Price: 793.0
199.79999999999995
Moving Average: 793.548
Last Price: 793.0
199.799999999999